## このノートブックは回路（bitファイル）を読み込む方法に関する記録です。

以下コードで回路（bitファイル）を読み込むことができる。
<pre>
```python
from pynq import Overlay
Overlay("./led_overlay.bit").download()
```
</pre>  
やってみる。

In [1]:
from pynq import Overlay
Overlay("/home/xilinx/pynq/bitstream/base.bit").download()
Overlay("./led_overlay.bit").download()
from pynq import PL
PL.ip_dict

#led_overlay.bitを読み込んだときに光るコマンド
from pynq import MMIO
led = MMIO(0x41200000,16)
led.write(0x0,0B0011)

光る。  
Overlay("./led_overlay.bit").download()について、Overlayクラスの中身を見てみると  
 "/sys/devices/soc0/amba/f8007000.devcfg/is_partial_bitstream"  
と  
 "/dev/xdevcfg"  
に書き込みをしていることがわかる。実際にこの２つのデバイスファイルに書き込むことで回路が書き換わるか試してみる。まず、基本の回路では光らないことを確認する。

In [2]:
from pynq import PL

#基本のOverlayを読み込む
from pynq import Overlay
Overlay("/home/xilinx/pynq/bitstream/base.bit").download()

#led_overlay.bitを読み込んだときに光るコマンド
from pynq import MMIO
led = MMIO(0x41200000,16)
led.write(0x0,0B1111)

光らない。２つのデバイスファイルに書き込みをすると。

In [3]:
import os
with open("./led_overlay.bit", 'rb') as f:
    buf = f.read()
         
        # Set is_partial_bitfile device attribute to 0
BS_IS_PARTIAL = "/sys/devices/soc0/amba/f8007000.devcfg/is_partial_bitstream"
with open(BS_IS_PARTIAL, 'w') as fd:
    fd.write('0')
         
        # Write bitfile to xdevcfg device
BS_XDEVCFG = "/dev/xdevcfg"
with open(BS_XDEVCFG, 'wb') as f:
    f.write(buf)

#led_overlay.bitを読み込んだときに光るコマンド
from pynq import MMIO
led = MMIO(0x41200000,16)
led.write(0x0,0B0011)

光る。一番最初のコードと同じ結果が得られた。　　

従い、回路（bitファイル）を読み込むためには、  
### "/sys/devices/soc0/amba/f8007000.devcfg/is_partial_bitstream"  
と  
### "/dev/xdevcfg"  
の２つのファイルを操作する必要があることがわかる。  

### 参考
Document  
http://pynq.readthedocs.io/en/latest/pynq.html#module-pynq.pl  
Overlayとpythonのやり取りについて網羅しているサイト。  
http://todotani.cocolog-nifty.com/blog/2017/01/pynq-z1overlayp.html  
xilinx/pynq/pl.pyのコード  
http://pynq.readthedocs.io/en/latest/_modules/pynq/pl.html  